#### Import Lib

In [24]:
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import KFold, train_test_split, GridSearchCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#### Đọc dữ liệu

In [15]:
df = pd.read_csv('../Data/input_data.csv')
df

,Brand,Processor_Name,Processor_Brand,RAM,RAM_TYPE,Display_type,GPU,GPU_Brand,SSD,OS,VRAM,Price_VND
0,HP,MediaTek Octa-core,MediaTek,4,DDR4 RAM,LED,Integrated Graphics,MediaTek,64,Google Chrome,0,6897000
1,Lenovo,AMD Hexa-Core Ryzen 5,AMD,8,DDR4 RAM,LCD,Integrated Graphics,AMD,512,Windows 11,0,10886700
2,Dell,Intel Core i5 (12th Gen),Intel,16,DDR5 RAM,LCD,GeForce RTX 3050,NVIDIA,512,Windows 11,4,23550000
3,HP,Intel Core i5 (12th Gen),Intel,8,DDR4 RAM,LCD,Integrated Graphics,Intel,512,Windows 11,0,16647000
4,Infinix,Intel Core i3 (11th Gen),Intel,8,LPDDR4X RAM,LCD,Integrated Graphics,Intel,512,Windows 11,0,6597000
...,...,...,...,...,...,...,...,...,...,...,...,...
3911,ASUS,Intel Core i5 (8th Gen),Intel,8,DDR4 RAM,LED,GeForce MX150,NVIDIA,512,Windows 10,2,25497000
3912,ASUS,Intel Core i3 (11th Gen),Intel,8,DDR4 RAM,LED,Integrated Graphics,Intel,256,Windows 11,0,12009300
3913,ASUS,AMD Octa-Core Ryzen 7,AMD,16,DDR4 RAM,LED,GeForce RTX 3050,NVIDIA,1024,Windows 11,4,22797000
3914,ASUS,AMD Quad-Core Ryzen 5,AMD,8,DDR4 RAM,LED,GeForce GTX 1650,NVIDIA,512,Windows 10,4,21572700


In [16]:
df.columns

Index(['Brand', 'Processor_Name', 'Processor_Brand', 'RAM', 'RAM_TYPE',
       'Display_type', 'GPU', 'GPU_Brand', 'SSD', 'OS', 'VRAM', 'Price_VND'],
      dtype='object')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3916 entries, 0 to 3915
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Brand            3916 non-null   object
 1   Processor_Name   3916 non-null   object
 2   Processor_Brand  3916 non-null   object
 3   RAM              3916 non-null   int64 
 4   RAM_TYPE         3916 non-null   object
 5   Display_type     3916 non-null   object
 6   GPU              3916 non-null   object
 7   GPU_Brand        3916 non-null   object
 8   SSD              3916 non-null   int64 
 9   OS               3916 non-null   object
 10  VRAM             3916 non-null   int64 
 11  Price_VND        3916 non-null   int64 
dtypes: int64(4), object(8)
memory usage: 367.3+ KB


#### Train - Test Split

In [18]:
# Giả sử dữ liệu của bạn có tên là df
X = df.drop('Price_VND', axis=1)  # Các biến độc lập
y = df['Price_VND']  # Biến mục tiêu

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Xác định biến số và biến phân loại
# Xác định các biến số và biến phân loại
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

#### Tạo Pipeline tiền xử lý

In [21]:
# Tạo pipeline cho tiền xử lý
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Kết hợp các bước tiền xử lý vào ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

#### Xây dựng Pipeline cho mô hình hồi quy

In [23]:
# Danh sách các mô hình hồi quy
models = {
    'XGBRegressor': XGBRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'KNeighborsRegressor': KNeighborsRegressor()
}

# Huấn luyện và đánh giá mô hình
results = {}

for model_name, model in models.items():
    # Tạo pipeline cho mô hình
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])
    
    # Huấn luyện mô hình
    pipeline.fit(X_train, y_train)
    
    # Dự đoán trên tập kiểm tra
    y_pred = pipeline.predict(X_test)
    
    # Đánh giá mô hình
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Lưu kết quả đánh giá
    results[model_name] = {'MAE': mae, 'MSE': mse, 'R2': r2}

# Hiển thị kết quả
results_df = pd.DataFrame(results).T
results_df


,MAE,MSE,R2
XGBRegressor,3.702117e+06,4.114120e+13,0.817971
DecisionTreeRegressor,4.295846e+06,5.479985e+13,0.757538
RandomForestRegressor,3.789520e+06,4.360828e+13,0.807055
LinearRegression,3.952835e+06,4.428475e+13,0.804062
Ridge,3.989306e+06,4.357127e+13,0.807219
KNeighborsRegressor,3.987239e+06,4.890782e+13,0.783608


Kết quả:
- Mô hình tốt nhất: XGBRegressor có R² cao nhất (81.8%) và MAE thấp, làm cho nó trở thành mô hình tốt nhất trong bộ so sánh này.
- Mô hình kém nhất: KNeighborsRegressor có R² thấp nhất (78.4%) và MSE cao nhất, cho thấy nó không phù hợp với dữ liệu này so với các mô hình khác.

Hướng phát triển:
-  Tối ưu tham số cho mô hình XGBRegressor

#### Tối ưu tham số cho mô hình XGBRegressor

In [36]:
# Danh sách một số param cần lưu ý
xgb_params = {
    'n_estimators',        # Số lượng cây quyết định
    'learning_rate',       # Tốc độ học (eta)
    'max_depth',             # Độ sâu tối đa của cây
    'min_child_weight',      # Trọng số tối thiểu của mỗi nút lá
    'subsample',           # Tỷ lệ mẫu huấn luyện mỗi cây
    'colsample_bytree',    # Tỷ lệ đặc trưng được chọn để huấn luyện mỗi cây
    'gamma',                 # Mức độ phạt để ngừng tạo nhánh
    'scale_pos_weight',      # Tỷ lệ giữa các lớp trong bài toán mất cân bằng
    'objective',  # Mục tiêu hồi quy
    'eval_metric',      # Chỉ số đánh giá
    'tree_method',      # Phương pháp xây dựng cây
    'lambda',              # Regularization L2 (Ridge)
    'alpha'               # Regularization L1 (Lasso)
}


In [40]:
# Xây dựng Pipeline riêng cho mô hình XGBRegressor
XGB_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor())  # Mô hình XGBRegressor
    ]
)

# Huấn luyện lại mô hình với tham số mặc định
XGB_pipeline.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình với tham số mặc định
y_pred = XGB_pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# In ra các kết quả đánh giá và tham số mặc định
print("Kết quả đánh giá với tham số mặc định:")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")
# XGB_pipeline.named_steps['regressor'].get_params()


# In ra các tham số có trong danh sách xgb_params
print("\nCác tham số của XGBRegressor trong danh sách cần lưu ý:")
xgb_model_params = XGB_pipeline.named_steps['regressor'].get_params()

for param in xgb_params:
    if param in xgb_model_params:
        print(f"{param}: {xgb_model_params[param]}")


Kết quả đánh giá với tham số mặc định:
MAE: 3702117.4776785714
MSE: 41141201332121.26
R²: 0.8179708923895523

Các tham số của XGBRegressor trong danh sách cần lưu ý:
objective: reg:squarederror
base_score: None
booster: None
callbacks: None
colsample_bylevel: None
colsample_bynode: None
colsample_bytree: None
device: None
early_stopping_rounds: None
enable_categorical: False
eval_metric: None
feature_types: None
gamma: None
grow_policy: None
importance_type: None
interaction_constraints: None
learning_rate: None
max_bin: None
max_cat_threshold: None
max_cat_to_onehot: None
max_delta_step: None
max_depth: None
max_leaves: None
min_child_weight: None
missing: nan
monotone_constraints: None
multi_strategy: None
n_estimators: None
n_jobs: None
num_parallel_tree: None
random_state: None
reg_alpha: None
reg_lambda: None
sampling_method: None
scale_pos_weight: None
subsample: None
tree_method: None
validate_parameters: None
verbosity: None


In [45]:
# Định nghĩa danh sách các tham số cần điều chỉnh
# param_grid = {
#     'regressor__n_estimators': [50, 100, 200],        # Số lượng cây quyết định
#     'regressor__learning_rate': [0.01, 0.05, 0.1],    # Tốc độ học (eta)
#     'regressor__max_depth': [3, 6, 10],                 # Độ sâu tối đa của cây
#     'regressor__min_child_weight': [1, 5, 10],          # Trọng số tối thiểu của mỗi nút lá
#     'regressor__subsample': [0.7, 0.8, 1.0],           # Tỷ lệ mẫu huấn luyện mỗi cây
#     'regressor__colsample_bytree': [0.7, 0.8, 1.0],    # Tỷ lệ đặc trưng được chọn để huấn luyện mỗi cây
#     'regressor__gamma': [0, 0.1, 0.2],                 # Mức độ phạt để ngừng tạo nhánh
#     'regressor__scale_pos_weight': [1, 10],             # Tỷ lệ giữa các lớp trong bài toán mất cân bằng
#     'regressor__tree_method': ['auto', 'hist'],         # Phương pháp xây dựng cây
#     'regressor__lambda': [0.5, 1.0],                    # Regularization L2 (Ridge)
#     'regressor__alpha': [0.0, 0.5],                     # Regularization L1 (Lasso)
# }
param_grid = {
    'regressor__n_estimators': [100, 200],          # Số lần lặp của quá trình boosting.
    'regressor__learning_rate': [0.01, 0.1],        # Tỉ lệ học
    'regressor__max_depth': [3, 6, 10],              # Độ sâu tối đa của cây
    'regressor__gamma': [0, 0.1],                   # Mức độ phạt để ngừng phân chia
    'regressor__colsample_bytree': [0.7, 0.8, 1.0], # Tỷ lệ cột chọn mẫu cho mỗi cây
    'regressor__reg_alpha': [0.1, 1, 10],           # Regularization L1
    'regressor__reg_lambda': [0.1, 1, 10]           # Regularization L2
}



# Xây dựng Pipeline với XGBRegressor
XGB_pipeline = Pipeline(
    steps=[('preprocessor', preprocessor),  # preprocessor là bước tiền xử lý dữ liệu
            ('regressor', XGBRegressor())]   # Mô hình XGBRegressor
)

# Sử dụng GridSearchCV để tối ưu tham số
grid_search = GridSearchCV(estimator=XGB_pipeline, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='r2')

# Huấn luyện GridSearchCV
grid_search.fit(X_train, y_train)

# Lấy ra tham số tối ưu và mô hình tối ưu
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Dự đoán và đánh giá mô hình tối ưu
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# In ra các kết quả đánh giá và tham số tối ưu
print("Kết quả đánh giá với tham số tối ưu:")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")

# In ra các tham số tối ưu
print("\nCác tham số tối ưu của XGBRegressor:")
for param, value in best_params.items():
    print(f"{param}: {value}")


Fitting 3 folds for each of 648 candidates, totalling 1944 fits
Kết quả đánh giá với tham số tối ưu:
MAE: 3769921.7391581633
MSE: 43368817973954.695
R²: 0.8081148100127227

Các tham số tối ưu của XGBRegressor:
regressor__colsample_bytree: 0.7
regressor__gamma: 0
regressor__learning_rate: 0.1
regressor__max_depth: 10
regressor__n_estimators: 100
regressor__reg_alpha: 1
regressor__reg_lambda: 10


In [ ]:
# Đánh giá mô hình với tham số tốt nhất
